In [1]:
import tensorflow.keras.backend as K
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Limpar o cache do TensorFlow-Keras
K.clear_session()

# Carregar o modelo MobileNetV2
modelo_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Camadas totalmente conectadas
camada = modelo_base.output
camada = GlobalAveragePooling2D()(camada)
prediction = Dense(1, activation='sigmoid')(camada)

construido_modelo = Model(inputs=modelo_base.input, outputs=prediction)

for layer in modelo_base.layers:
    layer.trainable = False

# Compilar o modelo
construido_modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Carregar os dados de treino
train_data = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_data.flow_from_directory(
    'treinar', 
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42)
start_time = time.time()
construido_modelo.fit(train_generator, epochs=150)
end_time = time.time()

# Medir o tempo de execução
execution_time = end_time - start_time
print('Tempo de execução: ', execution_time)

construido_modelo.save('my_model_binary.h5')


Found 15104 images belonging to 2 classes.
Epoch 1/150
472/472 [==============================] - 319s 671ms/step - loss: 0.7040 - accuracy: 0.5000
Epoch 2/150
472/472 [==============================] - 295s 624ms/step - loss: 0.6961 - accuracy: 0.5000
Epoch 3/150
472/472 [==============================] - 295s 625ms/step - loss: 0.6945 - accuracy: 0.5000
Epoch 4/150
472/472 [==============================] - 289s 611ms/step - loss: 0.6938 - accuracy: 0.5000
Epoch 5/150
472/472 [==============================] - 288s 609ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 6/150
472/472 [==============================] - 290s 613ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 7/150
472/472 [==============================] - 292s 617ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 8/150
472/472 [==============================] - 298s 631ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 9/150
472/472 [==============================] - 297s 629ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 10/150

In [3]:
import cv2
import os
import re
import numpy as np
from PIL import Image

def augmentData(imagens):
    for i, imagem in enumerate(imagens):
        # Abre a imagem como um array numpy
        img = np.array(Image.open("Raio-X/" + imagem))
        
       # Aplica a expressão regular ao nome da imagem
        match = re.search(r'\((\d+)\)', imagem)
        if match:
            number = int(match.group(1))

            # Obtém o número atual
            current_number = i + 1

            # Separa o nome do arquivo e a extensão
            filename = os.path.basename(imagem)
            filename, _ = os.path.splitext(filename)

           # new_filename = f"AkiTeste/{filename}_{current_number}"
           # new_filename2 = f"AkiTeste/{filename}_Espelhada_{current_number}"
           # new_filename3 = f"AkiTeste/{filename}_EspelhadaEQ_{current_number}"
            new_filename4 = f"Akitreinar/{filename}_{current_number}"
            #espelho = np.rot90(img, 2)
            #img_eq = equalizeHist(espelho)
            #img_eq2 = equalizeHist(img)
            #cv2.imwrite(f"{new_filename}.png", img)
            #cv2.imwrite(f"{new_filename2}.png", espelho)
           # cv2.imwrite(f"{new_filename3}.png", img_eq)
            cv2.imwrite(f"{new_filename4}.png", img)
        print(i)

def equalizeHist(img):
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img)
    return img_eq

# Diretório de destino para as imagens
pasta_destino = os.path.join(os.getcwd(), 'AkiTeste')
os.makedirs(pasta_destino, exist_ok=True)

# Nome do arquivo de texto contendo os nomes das imagens
nome_arquivo = 'nometeste.txt'

# Lê o arquivo de texto e remove as quebras de linha
with open(nome_arquivo, 'r') as arquivo:
    linhas = arquivo.read().splitlines()

# Executa a função para gerar as imagens aumentadas
augmentData(linhas)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
filename, extension = os.path.splitext(linhas[1])
print(filename)

FrightCC/f_right_cc (10)


In [31]:
def processar_imagens():
    # Diretório de origem das imagens
    diretorio_origem = "Akitreinar"

    # Diretório de destino para as imagens classificadas
    diretorio_destino = "treinar4"
    os.makedirs(diretorio_destino, exist_ok=True)

    # Mapeamento das classes BIRADS para a classificação binária
    classes_binarias = {"d": "I+II", "e": "I+II", "f": "III+IV", "g": "III+IV"}

    # Cria os subdiretórios para a classificação binária
    for classificacao in classes_binarias.values():
        subdiretorio = os.path.join(diretorio_destino, classificacao)
        os.makedirs(subdiretorio, exist_ok=True)

    # Percorre os arquivos do diretório de origem
    for arquivo in os.listdir(diretorio_origem):
        # Obtém o caminho completo do arquivo
        caminho_origem = os.path.join(diretorio_origem, arquivo)

        # Obtém a classe da imagem com base no primeiro caractere do nome do arquivo
        classe = arquivo.split("_")[0]

        # Verifica se a classe pertence à classificação binária
        if classe in classes_binarias:
            classificacao_binaria = classes_binarias[classe]

            # Copia o arquivo para o diretório de destino correspondente à classificação binária
            caminho_destino = os.path.join(diretorio_destino, classificacao_binaria, arquivo)
            shutil.copy2(caminho_origem, caminho_destino)

processar_imagens()


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

test_data = ImageDataGenerator(rescale=1./255)

test_generator = test_data.flow_from_directory(
    'treinar',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)



# Avaliar o modelo
predictions = construido_modelo.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)

test_labels = test_generator.classes

# Calcular as métricas
accuracy = accuracy_score(test_labels, predicted_classes)
precision = precision_score(test_labels, predicted_classes)
recall = recall_score(test_labels, predicted_classes)
f1 = f1_score(test_labels, predicted_classes)

cm = confusion_matrix(test_labels, predicted_classes)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print('Matriz de confusão: ', cm)
print('Acurácia: ', accuracy)
print('Precisão: ', precision)
print('Recall: ', recall)
print('F1 score: ', f1)
print('Sensibilidade: ', sensitivity)
print('Especificidade: ', specificity)

Found 15104 images belonging to 2 classes.
472/472 [==============================] - 235s 496ms/step
Matriz de confusão:  [[ 543 7009]
 [ 399 7153]]
Acurácia:  0.5095338983050848
Precisão:  0.5050840276797063
Recall:  0.947166313559322
F1 score:  0.6588376162844247
Sensibilidade:  0.947166313559322
Especificidade:  0.07190148305084745


In [10]:
import shutil

def processar_imagens4():
    # Diretório de origem das imagens
    diretorio_origem = "Akitreinar"

    # Diretório de destino para as imagens classificadas
    diretorio_destino = "test4"
    os.makedirs(diretorio_destino, exist_ok=True)

    # Mapeamento das classes BIRADS para a classificação binária
    classes_binarias = {"d": "I", "e": "II", "f": "III", "g": "IV"}

    # Cria os subdiretórios para cada classe distinta
    for classificacao in classes_binarias.values():
        subdiretorio = os.path.join(diretorio_destino, classificacao)
        os.makedirs(subdiretorio, exist_ok=True)

    # Percorre os arquivos do diretório de origem
    for arquivo in os.listdir(diretorio_origem):
        # Obtém o caminho completo do arquivo
        caminho_origem = os.path.join(diretorio_origem, arquivo)

        # Obtém a classe da imagem com base no primeiro caractere do nome do arquivo
        classe = arquivo.split("_")[0]

        # Verifica se a classe pertence às classes distingas
        if classe in classes_binarias:
            classificacao_distinta = classes_binarias[classe]

            # Copia o arquivo para o diretório de destino correspondente à classe distinta
            caminho_destino = os.path.join(diretorio_destino, classificacao_distinta, arquivo)
            shutil.copy2(caminho_origem, caminho_destino)

processar_imagens4()

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Limpar o cache do TensorFlow-Keras
K.clear_session()

# Carregar o modelo MobileNetV2
modelo_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Camadas totalmente conectadas
camada = modelo_base.output
camada = GlobalAveragePooling2D()(camada)
prediction = Dense(4, activation='sigmoid')(camada)

construido_modelo = Model(inputs=modelo_base.input, outputs=prediction)

for layer in modelo_base.layers:
    layer.trainable = False

# Compilar o modelo
construido_modelo.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Carregar os dados de treino
train_data = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_data.flow_from_directory(
    'test4', 
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42)
start_time = time.time()
construido_modelo.fit(train_generator, epochs=150)
end_time = time.time()

# Medir o tempo de execução
execution_time = end_time - start_time
print('Tempo de execução: ', execution_time)

construido_modelo.save('my_model_4.h5')


In [2]:
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image
import os
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

if os.name == 'posix':  # Linux ou macOS
    devnull = open('/dev/null', 'w')

os.dup2(devnull.fileno(), 2)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
model = tf.keras.models.load_model('my_model_4.h5')

# Carregar e pré-processar a imagem de entrada
img = image.load_img("/home/quebec/Documents/faculdade/pdISecreto/Processamento_e_An-lise_de_Imagens/PDI/App/Raio-X/R.png", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# Assuming preprocess_input is a function you've defined elsewhere or imported
x = preprocess_input(x)

# Fazer a classificação da imagem
predictions = model.predict(x)

# Definir a ordem das classes
classes = ["d", "e", "f", "g"]

indice_previsto = np.argmax(predictions)
classe_prevista = classes[indice_previsto]
print('Classe prevista:', classe_prevista)

            # Printar as probabilidades de cada classe
print("Probabilidades:")
for i, classe in enumerate(classes):
    print(f"{classe}: {predictions[0][i]*100:.2f}%")


2023-06-08 09:32:45.629528: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 09:32:45.805656: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 09:32:48.624114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1/1 [==============================] - 3s 3s/step
Classe prevista: d
Probabilidades:
d: 99.82%
e: 0.18%
f: 0.91%
g: 0.76%
